In [ ]:
import pickle
import pandas as pd
# open_data = pd.read_csv('open_data.csv', index_col=0, parse_dates=True)
with open('optimal_pairs_by_semester.pkl', 'rb') as f:
    optimal_pairs_by_semester = pickle.load(f)
with open('open_bysemester.pkl', 'rb') as f:
    open_bysemester = pickle.load(f)    
with open('new_result_dict.pickle', 'rb') as file:
    new_result_dict = pickle.load(file)
with open('riskfree_bysemester.pkl', 'rb') as file:
    riskfree_bysemester = pickle.load(file)

In [ ]:
# Assuming optimal_pairs_by_semester is your dictionary
limited_optimal_pairs = {}

for semester, pairs_array in optimal_pairs_by_semester.items():
    # Limit to the first 30 elements
    limited_pairs = pairs_array[:30]
    # Store the limited pairs in the new dictionary
    limited_optimal_pairs[semester] = limited_pairs

# limited_optimal_pairs now contains the first 30 pairs for each semester

In [ ]:
pair_results = {}
# Iterate through semesters
for i in range(1, len(optimal_pairs_by_semester) + 1):
    semester_pairs = optimal_pairs_by_semester['Semester_' + str(i)]
    open_prices_next_semester = open_bysemester['Semester_' + str(i+1)] # del +1
    
    semester_pair_data = {}

    # Iterate through pairs in the current semester
    for idx, pair in enumerate(semester_pairs):
        security_0_open = open_prices_next_semester[pair[0]]
        security_1_open = open_prices_next_semester[pair[1]]
        
        pair_data = pd.DataFrame({
            'open0': security_0_open,
            'open1': security_1_open
        })
        
        # Store the DataFrame for the pair in the semester_pair_data dictionary
        semester_pair_data[idx] = pair_data
    
    # Store the dictionary of DataFrames for the semester in the pair_results dictionary
    pair_results['Semester_' + str(i+1)] = semester_pair_data #del +1

In [ ]:
import statsmodels.api as sm
def calc_zscore(spread):
    zscore = (spread - np.mean(spread))/np.std(spread)
    return zscore
def calc_norm_spread(pair, lookback_dict, i):
    security_0 = pair[0]
    security_1 = pair[1]

    end_index = len(open_bysemester['Semester_' + str(e)])  +i
    start_index = i # + int(len(open_bysemester['Semester_' + str(e)]) * 2/3)
    window_data0 = lookback_dict[start_index:end_index][security_0]
    window_data1 = lookback_dict[start_index:end_index][security_1]

    x = sm.add_constant(np.asarray(window_data0))
    y = np.asarray(window_data1)

    model = sm.OLS(y, x)
    result = model.fit()

    alpha, beta = result.params[0], result.params[1]

    spread_res = y - (alpha + beta * x.T[1])
    
    norm_spread = calc_zscore(spread_res)

    return pd.Series(norm_spread), beta

def pnl(risk_free, prices_dict, optimal_pairs, lookback_dict, closelong, closeshort, sellth, buyth, fee):
    last_cum_returns = []
    pair_returns = []
    pair_daily_returns = []
    """
prices_dict contains open0 and open1, lookback_dict is the same as prices_dict but also includes the
previous semester. Risk_free is the daily 3-month t-bill rate
    """
    for pair, df in zip(optimal_pairs, prices_dict.values()):
        signals0 = np.zeros(len(df))
        pos1 = np.zeros(len(df))
        pos0 = np.zeros(len(df))
        days_since_trade = 0
        beta = 0
        
        for i in range(0, len(df)): 
            if i < 1:
                prev_signal = 0
            else:
                prev_signal = signals0[i - 1]

            norm_spread, curr_beta = calc_norm_spread(pair, lookback_dict, i)
            curr_spread = norm_spread.iloc[-1]  # Extract the last value of the Series

            if days_since_trade % 6 == 0:
                # Update beta every 5 days
                beta = curr_beta
            if (curr_spread >= sellth and prev_signal == 0) or (curr_spread <= buyth and prev_signal == 0):
                # Enter trade at current_beta
                signals0[i] = -1 if curr_spread >= sellth else 1
                days_since_trade = 1  # Reset the days since trade
                beta = curr_beta  # Set beta to current beta
            elif closelong < curr_spread and prev_signal == 1:
                signals0[i] = 0
            elif closeshort > curr_spread and prev_signal == -1:
                signals0[i] = 0
            else:
                signals0[i] = prev_signal
            
            if days_since_trade > 0:
                days_since_trade += 1
            
            if abs(beta) <= 1:
                pos0[i] = -signals0[i] # * beta 
                pos1[i] = signals0[i] 
            else:
                pos0[i] = -signals0[i]
                pos1[i] = signals0[i] #/ beta
                
        df['pos0'] = pos0
        df['pos1'] = pos1
        
        pos0[-1] = 0
        pos1[-1] = 0
        
        df['pos0_diff'] = df['pos0'].diff().abs()
        df['pos1_diff'] = df['pos1'].diff().abs() # units buyed or selled for transactions cost
        
        df['logret_stock0'] = np.log(df['open0'] / df['open0'].shift(1))
        df['logret_stock1'] = np.log(df['open1'] / df['open1'].shift(1))
        
        df['logret_leg0'] = df['logret_stock0'] * df['pos0'].shift(1) - (df['pos0_diff'] * fee) 
        df['logret_leg1'] = df['logret_stock1'] * df['pos1'].shift(1) - (df['pos1_diff'] * fee)

        df['cumulative_return0'] = df['logret_leg0'].dropna().cumsum().apply(np.exp)
        df['cumulative_return1'] = df['logret_leg1'].dropna().cumsum().apply(np.exp)
 #       print(df['cumulative_return1'], df['cumulative_return0'])
        last_cumulative_return0 = df['cumulative_return0'].iloc[-1] -1  # Get the last value
        last_cumulative_return1 = df['cumulative_return1'].iloc[-1] -1
        
        total_return = (last_cumulative_return0 + last_cumulative_return1) *100
        pair_returns.append((pair,total_return))
        
        pair_ret = (np.exp(df['logret_leg0']) + 
                    np.exp(df['logret_leg1']))
        pair_daily_returns.append(pair_ret)
        
    Final_returns = pd.DataFrame(pair_returns, columns=['Pair', 'Last_Cumpair_Return'])
    freturn =  Final_returns['Last_Cumpair_Return'].mean() 
    
    pair_daily_returnsf = pd.concat(pair_daily_returns, axis=1)

    weights = np.array([1 / len(optimal_pairs)] * len(optimal_pairs))
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(pair_daily_returnsf.cov(), weights)))
    
    rfrate = risk_free.mean() 
  #  portfolio_excess_return = freturn*2 - rfrate # annualize without coumpounding
    annualized_portfolio_excess_return = (((1+freturn/100) ** 2) - 1)*100 - rfrate
    sharpe_ratio = annualized_portfolio_excess_return / (portfolio_volatility*np.sqrt(252)*100)
  #  print(Final_returns)

    return freturn, portfolio_volatility, sharpe_ratio

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfs0bpbeta = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=2, buyth=-2, 
        fee=0)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfs0bpbeta.append(semester_df)

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfs100bpopttimi = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=1.8, buyth=-2.2, 
        fee=0.01)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfs100bpopttimi.append(semester_df)

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfs100bpdollaropttimi = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=1.8, buyth=-2.2, 
        fee=0.01)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfs100bpdollaropttimi.append(semester_df)

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs100bpdollaropttimi, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}")

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs100bpoptimized, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}")

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs50bpbeta, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}")

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs100bpopttimi, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}")

In [ ]:
from matplotlib.font_manager import FontProperties
custom_font = FontProperties(fname=r'C:\Users\usuario\AppData\Local\Microsoft\Windows\Fonts\cmu.serif-roman.ttf')

In [ ]:
portfolio_returns100bp = [df['Portfolio_Return'].values/100 for df in portfolio_dfs100bpbeta]
portfolio_returns50bp = [df['Portfolio_Return'].values/100 for df in portfolio_dfs50bpbeta]
portfolio_returns100bpdollar = [df['Portfolio_Return'].values/100 for df in portfolio_dfs100bp]
portfolio_returns50bpdollar = [df['Portfolio_Return'].values/100 for df in portfolio_dfs50bp]
portfolio_returns100bpoptimized = [df['Portfolio_Return'].values/100 for df in portfolio_dfsbeta100bpoptimized]

In [ ]:
returns100bp = [ret + 1 for ret in portfolio_returns100bp]
cumulative_product100bp = [1]  # Initialize the cumulative product list with 1 as the first element

for ret in returns100bp:
    cumulative_product100bp.append(cumulative_product100bp[-1] * ret)

returns50bp = [ret + 1 for ret in portfolio_returns50bp]
cumulative_product50bp = [1]  # Initialize the cumulative product list with 1 as the first element

for ret in returns50bp:
    cumulative_product50bp.append(cumulative_product50bp[-1] * ret)

returns100bpdollar = [ret + 1 for ret in portfolio_returns100bpdollar]
cumulative_product100bpdollar = [1]  # Initialize the cumulative product list with 1 as the first element

for ret in returns100bpdollar:
    cumulative_product100bpdollar.append(cumulative_product100bpdollar[-1] * ret)

returns50bpdollar = [ret + 1 for ret in portfolio_returns50bpdollar]
cumulative_product50bpdollar = [1]  # Initialize the cumulative product list with 1 as the first element

for ret in returns50bpdollar:
    cumulative_product50bpdollar.append(cumulative_product50bpdollar[-1] * ret)

returns100bpoptimized = [ret + 1 for ret in portfolio_returns100bpoptimized]
cumulative_product100bpoptimized = [1]  # Initialize the cumulative product list with 1 as the first element

for ret in returns100bpoptimized:
    cumulative_product100bpoptimized.append(cumulative_product100bpoptimized[-1] * ret)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# Clear the current figure
plt.clf()

# Create a list of corresponding years for the x-axis
years = list(range(2001, 2024))

# Create a list of x-values by repeating each year twice to match the number of data points
# Exclude the last year to match the data size
x_values = np.repeat(years, 2)

# Combine the first element with the concatenated arrays
cumulative_product50bp_combined = [cumulative_product50bp[0]] + list(np.concatenate(cumulative_product50bp[1:]))
cumulative_product100bp_combined = [cumulative_product100bp[0]] + list(np.concatenate(cumulative_product100bp[1:]))
cumulative_product50bpdollar_combined = [cumulative_product50bpdollar[0]] + list(np.concatenate(cumulative_product50bpdollar[1:]))
cumulative_product100bpdollar_combined = [cumulative_product100bpdollar[0]] + list(np.concatenate(cumulative_product100bpdollar[1:]))
cumulative_product100bpoptimized_combined = [cumulative_product100bpoptimized[0]] + list(np.concatenate(cumulative_product100bpoptimized[1:]))
# Create subplots for better visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the Portfolio_Return series for each list
ax.plot(x_values, cumulative_product50bpdollar_combined, label='Dollar-neutral 50bp ts. costs')
ax.plot(x_values, cumulative_product50bp_combined, label='Beta-neutral 50bp ts. costs')
ax.plot(x_values, cumulative_product100bpoptimized_combined, label='Dollar-neutral 100bp ts. costs optimized')
ax.plot(x_values, cumulative_product100bpdollar_combined, label='Dollar-neutral 100bp ts. costs')
ax.plot(x_values, cumulative_product100bp_combined, label='Beta-neutral 100bp ts. costs')
legend_font = FontProperties(fname=r'C:\Users\usuario\AppData\Local\Microsoft\Windows\Fonts\cmu.serif-roman.ttf', size=16)  # Replace custom_font_path with your font file path

# Set font properties for the legend
# Set labels and title
ax.set_xlabel('Year', fontproperties=custom_font, fontsize=17)
ax.set_ylabel('Performance', fontproperties=custom_font, fontsize=17)
ax.set_title('Equity curves', fontproperties=custom_font, fontsize=17)
ax.legend(prop=legend_font)
for tick in ax.get_xticklabels():
    tick.set_fontproperties(legend_font)
for tick in ax.get_yticklabels():
    tick.set_fontproperties(legend_font)

# Show the plot
plt.grid(True)
plt.savefig('equitycurvessou.png', dpi=800)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
# for s-scores we need to do
optimal_pairs_by_semester['Semester_35'] = [pair for pair in optimal_pairs_by_semester['Semester_35'] if pair != ('ASR', 'CIVI')]
np.array(optimal_pairs_by_semester['Semester_35'])
type(optimal_pairs_by_semester['Semester_35'])
pair_results = {}
# Iterate through semesters
for i in range(1, len(optimal_pairs_by_semester) + 1):
    semester_pairs = optimal_pairs_by_semester['Semester_' + str(i)]
    open_prices_next_semester = open_bysemester['Semester_' + str(i+1)] # del +1
    
    semester_pair_data = {}

    # Iterate through pairs in the current semester
    for idx, pair in enumerate(semester_pairs):
        security_0_open = open_prices_next_semester[pair[0]]
        security_1_open = open_prices_next_semester[pair[1]]
        
        pair_data = pd.DataFrame({
            'open0': security_0_open,
            'open1': security_1_open
        })
        
        # Store the DataFrame for the pair in the semester_pair_data dictionary
        semester_pair_data[idx] = pair_data
    
    # Store the dictionary of DataFrames for the semester in the pair_results dictionary
    pair_results['Semester_' + str(i+1)] = semester_pair_data #del +1

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import math
def calc_ou_par(pair, result_dict, i):
    security_0 = pair[0]
    security_1 = pair[1]

    end_index = len(open_bysemester['Semester_' + str(e)])  +i
    start_index = i #+ int(len(open_bysemester['Semester_' + str(e)]) * 2/3)
    window_data0 = result_dict[start_index:end_index][security_0].pct_change().dropna()
    window_data1 = result_dict[start_index:end_index][security_1].pct_change().dropna()

  
    x = sm.add_constant(np.asarray(window_data0))
    y = np.asarray(window_data1)

    # Get parameters and calculate spread
    model = sm.OLS(y, x)
    result = model.fit()

    alpha, beta = result.params[0], result.params[1]

    res = y - (alpha + beta * x.T[1])
    cumres = res.cumsum()
    cumres_series = pd.Series(cumres)
    x_cumres = sm.add_constant(np.asarray(cumres_series.shift(1).dropna()))
    y_cumres = cumres_series[1:].reset_index(drop=True)
    model_cumres = sm.OLS(y_cumres, x_cumres)
    result_cumres = model_cumres.fit()
    a, b = result_cumres.params[0], result_cumres.params[1]

    aux_residuals = result_cumres.resid
    var_auxres = np.var(aux_residuals)
    
    # Calculate kappa and m
    kappa = -math.log(b) * len(open_bysemester['Semester_' + str(e)])
    m = a / (1 - b)

    sigma = math.sqrt(var_auxres * 2 * kappa / (1 - b**2))
    sigma_eq = math.sqrt(var_auxres / (1 - b**2))
    s = -m / sigma_eq
    s_mod = s - a/(kappa*sigma_eq) 

    return s_mod, beta


def pnl(risk_free, prices_dict, optimal_pairs, lookback_dict, closelong, closeshort, sellth, buyth, fee):
    last_cum_returns = []
    pair_returns = []
    pair_daily_returns = []
    """

    """
    for pair, df in zip(optimal_pairs, prices_dict.values()):
        signals0 = np.zeros(len(df))
        pos1 = np.zeros(len(df))
        pos0 = np.zeros(len(df))
        days_since_trade = 0
        beta = 0
        
        for i in range(0, len(df)): 
            if i < 1:
                prev_signal = 0
            else:
                prev_signal = signals0[i - 1]

            s_score, curr_beta = calc_ou_par(pair, lookback_dict, i)
            
         #   if i==0:
          #      s_score=0
           # else:
            #    s_score=s_score

            if days_since_trade % 6 == 0:
                # Update beta every 5 days
                beta = curr_beta
            if (s_score >= sellth and prev_signal == 0) or (s_score <= buyth and prev_signal == 0):
                # Enter trade at current_beta
                signals0[i] = -1 if s_score >= sellth else 1
                days_since_trade = 1  # Reset the days since trade
                beta = curr_beta  # Set beta to current beta
            elif closelong < s_score and prev_signal == 1:
                signals0[i] = 0
            elif closeshort > s_score and prev_signal == -1:
                signals0[i] = 0
            else:
                signals0[i] = prev_signal
            
            if days_since_trade > 0:
                days_since_trade += 1
            
            if abs(beta) <= 1:
                pos0[i] = -signals0[i] * beta 
                pos1[i] = signals0[i] 
            else:
                pos0[i] = -signals0[i]
                pos1[i] = signals0[i] / beta
                
        df['pos0'] = pos0
        df['pos1'] = pos1
        
        pos0[-1] = 0
        pos1[-1] = 0
        
        df['pos0_diff'] = df['pos0'].diff().abs()
        df['pos1_diff'] = df['pos1'].diff().abs() # units buyed or selled for transactions cost
        
        df['logret_stock0'] = np.log(df['open0'] / df['open0'].shift(1))
        df['logret_stock1'] = np.log(df['open1'] / df['open1'].shift(1))
        
        df['logret_leg0'] = df['logret_stock0'] * df['pos0'].shift(1) - (df['pos0_diff'] * fee) 
        df['logret_leg1'] = df['logret_stock1'] * df['pos1'].shift(1) - (df['pos1_diff'] * fee)

        df['cumulative_return0'] = df['logret_leg0'].dropna().cumsum().apply(np.exp)
        df['cumulative_return1'] = df['logret_leg1'].dropna().cumsum().apply(np.exp)
 #       print(df['cumulative_return1'], df['cumulative_return0'])
        last_cumulative_return0 = df['cumulative_return0'].iloc[-1] -1  # Get the last value
        last_cumulative_return1 = df['cumulative_return1'].iloc[-1] -1
        
        total_return = (last_cumulative_return0 + last_cumulative_return1) *100
        pair_returns.append((pair,total_return))
        
        pair_ret = (np.exp(df['logret_leg0']) + 
                    np.exp(df['logret_leg1']))
        pair_daily_returns.append(pair_ret)
        
    Final_returns = pd.DataFrame(pair_returns, columns=['Pair', 'Last_Cumpair_Return'])
    freturn =  Final_returns['Last_Cumpair_Return'].mean() 
    pd.set_option('display.max_rows', None)
    
    pair_daily_returnsf = pd.concat(pair_daily_returns, axis=1)

    weights = np.array([1 / len(optimal_pairs)] * len(optimal_pairs))
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(pair_daily_returnsf.cov(), weights)))
    
    rfrate = risk_free.mean() 
  #  portfolio_excess_return = freturn*2 - rfrate # annualize without coumpounding
    annualized_portfolio_excess_return = (((1+freturn/100) ** 2) - 1)*100 - rfrate
    sharpe_ratio = annualized_portfolio_excess_return / (portfolio_volatility*np.sqrt(252)*100)
  #  print(Final_returns)

    return freturn, portfolio_volatility, sharpe_ratio

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfs50bpbetaoptimized = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=1.8, buyth=-2.2, 
        fee=0.005)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfs50bpoptimized.append(semester_df)

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfs100bpbetaoptimized = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=1.8, buyth=-2.2, 
        fee=0.01)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfs100bpoptimized.append(semester_df)

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfsbeta100bpoptimized = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=1.8, buyth=-2.2, 
        fee=0.01)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfsbeta100bpoptimized.append(semester_df)

In [ ]:
import pandas as pd
import numpy as np
# Create an empty list to store DataFrames for each semester
portfolio_dfs50bp = []

# Loop through the backtest results by semester
for e in range(1,  len(optimal_pairs_by_semester) - 1):
    prices_dict = pair_results['Semester_' + str(e + 1)]
    lookback_dict = new_result_dict['Semester_' + str(e)]
    optimal_pairs = optimal_pairs_by_semester['Semester_' + str(e)]
    risk_free = riskfree_bysemester['Semester_' + str(e+1)]
    
    # Call the pnl function for the current semester and pairs
    freturn, dvol, sharpe_ratio = pnl(
        risk_free, prices_dict, optimal_pairs, lookback_dict, closelong=0, closeshort=0, sellth=2, buyth=-2, 
        fee=0.005)
    
    # Create a DataFrame for the current semester and portfolio return
    semester_df = pd.DataFrame({'Semester': ['Semester_' + str(e + 1)], 
                                'Portfolio_Return': [freturn],
                                'dvol': [dvol],
                                'sharpe': [sharpe_ratio]})
    
    # Append the DataFrame to the list
    portfolio_dfs50bp.append(semester_df)

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs50bpoptimized, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}") 

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfsbeta100bpoptimized, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}") 

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs50bp, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}") 

In [ ]:
import pandas as pd
combined_portfolio100bp = pd.concat(portfolio_dfs100bpbeta, ignore_index=True)

# Calculate the mean portfolio return, mean portfolio volatility, and mean Sharpe ratio
mean_portfolio_return = combined_portfolio100bp['Portfolio_Return'].mean()
mean_dvol = combined_portfolio100bp['dvol'].mean()
mean_sharpe_ratio = combined_portfolio100bp['sharpe'].mean()

print(f"Mean portfolio return: {mean_portfolio_return:.4f}")
print(f"Mean daily volatility: {mean_dvol:.4f}") 
print(f"Mean sharpe ratio: {mean_sharpe_ratio:.4f}") 

In [ ]:
portfolio_dfs100bpoptimized